In [ ]:
from numpy import loadtxt
import string
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import os
import glob
from matplotlib import cm
from PIL import Image, ImageOps
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import cv2


<h2> Auxiliary functions </h2>

In [ ]:
import numpy

def gaussian2(size, sigma):
    """Returns a normalized circularly symmetric 2D gauss kernel array
    
    f(x,y) = A.e^{-(x^2/2*sigma^2 + y^2/2*sigma^2)} where
    
    A = 1/(2*pi*sigma^2)
    
    as define by Wolfram Mathworld 
    http://mathworld.wolfram.com/GaussianFunction.html
    """
    A = 1/(2.0*numpy.pi*sigma**2)
    x, y = numpy.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = A*numpy.exp(-((x**2/(2.0*sigma**2))+(y**2/(2.0*sigma**2))))
    return g

def fspecial_gauss(size, sigma):
    """Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x, y = numpy.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = numpy.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

In [ ]:
import sys
import numpy
from scipy import signal
from scipy import ndimage
def ssim(img1, img2, cs_map=False):
    """Return the Structural Similarity Map corresponding to input images img1 
    and img2 (images are assumed to be uint8)
    
    This function attempts to mimic precisely the functionality of ssim.m a 
    MATLAB provided by the author's of SSIM
    https://ece.uwaterloo.ca/~z70wang/research/ssim/ssim_index.m
    """
    img1 = img1.astype(numpy.float64)
    img2 = img2.astype(numpy.float64)
    size = 11
    sigma = 1.5
    window = fspecial_gauss(size, sigma)
    K1 = 0.01
    K2 = 0.03
    L = 255 #bitdepth of image
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = signal.fftconvolve(window, img1, mode='valid')
    mu2 = signal.fftconvolve(window, img2, mode='valid')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = signal.fftconvolve(window, img1*img1, mode='valid') - mu1_sq
    sigma2_sq = signal.fftconvolve(window, img2*img2, mode='valid') - mu2_sq
    sigma12 = signal.fftconvolve(window, img1*img2, mode='valid') - mu1_mu2
    if cs_map:
        return (((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2)), 
                (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2))
    else:
        return ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2))

def msssim(img1, img2):
    """This function implements Multi-Scale Structural Similarity (MSSSIM) Image 
    Quality Assessment according to Z. Wang's "Multi-scale structural similarity 
    for image quality assessment" Invited Paper, IEEE Asilomar Conference on 
    Signals, Systems and Computers, Nov. 2003 
    
    Author's MATLAB implementation:-
    http://www.cns.nyu.edu/~lcv/ssim/msssim.zip
    """
    level = 5
    weight = numpy.array([0.0448, 0.2856, 0.3001, 0.2363, 0.1333])
    downsample_filter = numpy.ones((2, 2))/4.0
    im1 = img1.astype(numpy.float64)
    im2 = img2.astype(numpy.float64)
    mssim = numpy.array([])
    mcs = numpy.array([])
    for l in range(level):
        ssim_map, cs_map = ssim (im1, im2, cs_map=True)
        mssim = numpy.append (mssim, ssim_map.mean())
        mcs = numpy.append (mcs, cs_map.mean())
        filtered_im1 = ndimage.filters.convolve(im1, downsample_filter, 
                                                mode='reflect')
        filtered_im2 = ndimage.filters.convolve(im2, downsample_filter, 
                                                mode='reflect')
        im1 = filtered_im1[::2, ::2]
        im2 = filtered_im2[::2, ::2]
    #print (mcs[0:level-1]**2)
    #print (weight[0:level-1]) 
    return (numpy.prod(mcs[0:level-1]**weight[0:level-1])*
                    (mssim[level-1]**weight[level-1]))


In [ ]:
import math
def make_grid (count, index, cols):
    if (count >= 10):
        #index_col = (index % cols) + 1
        #index_row = (index / cols) + 1
        total_rows = math.ceil(count / cols)
        total_cols = cols
        #options = str (total_rows) + str (total_cols) + 
        return total_rows,total_cols
    else:
        total_rows = math.ceil(count / cols)
        total_cols = cols
        #options = str (total_rows) + str (total_cols) + 
        return total_rows,total_cols    

In [ ]:
def ssim_plot_images (original_image, images):
    maps = []
    original_data = numpy.asarray (original_image)
    fil_data = []
    import pylab
    try:
        from PIL import Image
        for image in images:    
            img = numpy.asarray(image)
            fil_data.append (img)
    except Exception (e):
        e = 'Cannot load images' + str(e)
        print >> sys.stderr, e
    ssimRes = []
    msSimRes = []
    best_index = -1
    best = 0
    for ind, d in enumerate(fil_data):
        original_data = (original_data/original_data.max())*255
        d = (d/d.max()) * 255
        ssim_map = ssim(original_data, d)
        ms_ssim = msssim(original_data, d)
        if (ms_ssim > best):
            best = ms_ssim
            best_index = ind
        #print (original_data [original_data != 0].shape)
        ssimRes.append (ssim_map)
        msSimRes.append (ms_ssim)
    pylab.figure(figsize=(8, 6), dpi=300)
    #options = '1' + str (len (paths)) + '1'
    r, c = make_grid (len (images) + 1, 1, 3)
    #print (r, ' ', c)
    pylab.subplot(r, c , 1)
    plt.axis('off')
    pylab.title('Original')
    pylab.imshow(original_data, interpolation='nearest', cmap = 'RdBu_r')
    for i in range (len (images)):    
        #options = '1' + str (len (paths)) + str (i + 2)
        pylab.subplot(r, c, i + 2)
        pylab.imshow(fil_data [i], interpolation='nearest', cmap = 'RdBu_r')
        #pylab.subplot(133)
        s = "{value:.4f}"
        s = s.format(value = msSimRes[i])
        title = s
        pylab.title(s)
        #pylab.title('SSIM Map\n SSIM: %f\n MSSSIM: %f' % (ssimRes[i].mean(), msSimRes[i]))
        plt.axis('off')
        #pylab.imshow(ssim_map, interpolation='nearest')
    rand_ind = np.random.randint(1000000)    
    filename = original_path.split ('/')[-1].replace ('.fits','.png')
    filename = str (rand_ind) + '.png'
    pylab.savefig('datasets/msim_paper2/' + filename,bbox_inches='tight',pad_inches=0)
    return best_index
    #pylab.show()

In [ ]:
def ssim_plot (original_path, paths, options):
    maps = []
    original_data = fits.open (original_path)
    original_data = numpy.asarray (original_data[0].data)
    fil_data = []
    import pylab
    try:
        from PIL import Image
        for path in paths:    
            data = fits.open(path)
            fil_map = data[0].data
            img = numpy.asarray(fil_map)
            fil_data.append (fil_map)
    except Exception (e):
        e = 'Cannot load images' + str(e)
        print >> sys.stderr, e
    ssimRes = []
    msSimRes = []
    best_index = -1
    best = 0
    for ind, d in enumerate(fil_data):
        original_data = (original_data/original_data.max())*255
        d = (d/d.max()) * 255
        ssim_map = ssim(original_data, d)
        ms_ssim = msssim(original_data, d)
        if (ms_ssim > best):
            best = ms_ssim
            best_index = ind
    return best_index

In [ ]:
def find_best_msim_maps (): 
    i = 0
    best_msims = []
    for key, v in filaments.items ():
        original_path = 'datasets/colden/' + key + '.fits'     
        items = [k for k,v1 in v]
        mask = fits.open (items [1])
        opts = [v1 for k,v1 in v]
        best_msims.append (ssim_plot (original_path, items, opts))
        i+=1
    return best_msims  

<h1> reading fits files </h1>

<h2> Reading Planck dataset </h2>

<h3> Reading Planck-1 dataset </h3>

In [ ]:
import os
import glob
from matplotlib import cm
from PIL import Image, ImageOps
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import tensorflow as tf
images_list = [f for f in glob.glob("datasets/Pdata/*_I_*")]
rht_list = [f for f in glob.glob("datasets/Pdata/*_SUPERHT_THETA_*")]
mask_list = [f for f in glob.glob("datasets/Pdata/*_SUPERHT_G*")]
images_all = []
angles_all = []
masks_all = []
pil_all =  []
for index_img, img in enumerate (images_list):
    image_f = fits.open(img)
    image_d = image_f [0].data
    newSize = (128,128)
    image_d = Image.fromarray((image_d / image_d.max()) * 255)
    image_d = image_d.resize(newSize)
    image_d = np.array (image_d)
    
    ind = img.index ('_I_')
    angles_file = img[:ind] + '_SUPERHT_THETA_' + img [ind + 3:]
    mask_file = img[:ind] + '_SUPERHT_' + img [ind + 3:]
    print (angles_file)
    angles = fits.open(angles_file)
    angles = angles[0].data
    fils = fits.open(mask_file)
    fils = fils[0].data
    
    angles = Image.fromarray(angles)
    angles = angles.resize(newSize)
    angles = np.array (angles)
    
    fils = np.array (fils)
    fils = Image.fromarray((fils / fils.max()) * 255)
    fils = fils.resize(newSize)
    
    images_all.append (np.array (image_d))
    angles_all.append (np.array (angles))
    masks_all.append (np.array (fils))

<h3> Reading Planck-cc dataset </h3>

In [ ]:
import os
import glob
from matplotlib import cm
from PIL import Image, ImageOps
from skimage import data, color
import subprocess
from skimage.transform import rescale, resize, downscale_local_mean

def find_masks_and_images_best ():
    images_all = []
    masks_all = []
    pil_all =  []
    i = 0
    filament_names = list (set ([d.split ('\\')[1].split ('lon')[-1][0:-5].replace ('_smr27_wlen7','') for d in glob.glob ('cc_fil/BEST/*')]))
    root = os.getcwd().replace ('\\','/') + '/'
    for filament in filament_names:
        orig_path = root + 'cc_fil/BEST/_I_g_lon' + filament + '.fits'
        angle_path = root + 'cc_fil/BEST/theta/suprht_theta_lon-' + filament + '_K27_BAR7.fits'
        angle_path2 = root + 'cc_fil/BEST/theta/suprht_theta_lon' + filament + '_K27_BAR7.fits'
        if (os.path.exists (angle_path) == False):
            if (os.path.exists (angle_path2) == False):
                continue
            else:
                angle_path = angle_path2
        
        if (os.path.exists (orig_path) == False):
            continue    
        mask = fits.open (angle_path)
        image_f = fits.open (orig_path)
        print (angle_path)
        image_d = np.array (image_f[0].data).astype('float32')
        image_dd = (image_d / image_d.max()) *  255.0;
        image_d = Image.fromarray(image_dd)
        newSize = (128,128)
        image_p = image_d.resize(newSize)
        image_f = np.array (image_p).astype ('float32')
        image_f = (image_f / image_f.max()) * 255.0;
        image_f = image_f.clip(min = 0)
        image_dd = image_f[image_f != 0]
        mask_d = np.array (mask[0].data)
        mask_p = Image.fromarray(mask_d)
        newSize = (128,128)
        mask_p = mask_p.resize(newSize)
        mask = np.array (mask_p)
        mask = mask.tolist()
        masks_all.append (mask)
        images_all.append (image_f)
        i+=1
    return images_all, masks_all

<h2> Herschel dataset </h2>

In [ ]:
import os
import glob
dirs = [d.split ('\\')[1] for d in glob.glob ('SUPRHT/OUTPUT*COLD_DEN')]
print (dirs)
filaments = {}
for d in dirs:
    cur_folder = 'SUPRHT/' + d
    files = [f.split ('\\')[1] for f in glob.glob (cur_folder+'/*') if f.count ('THETA') == 0]
    for file in files:
        fil = file.split ('_SUPRHT_')[0]
        if (fil in filaments):
            path = cur_folder + '/' + file
            opt = file.split ('_') [-2:]
            opt[1] = opt[1].replace ('.fits','')
            opt = ' '.join (opt)
            data = fits.open(path)
            filaments[fil].append ((path,opt))
        else:
            path = cur_folder + '/' + file
            data = fits.open(path)
            fil_map = data[0].data 
            filaments[fil] = []
            opt = file.split ('_') [-2:]
            opt[1] = opt[1].replace ('.fits','')
            opt = ' '.join (opt)
            filaments[fil].append ((path,opt))

<h2> FInding best parameters of RHT with MSSIM for each Herschel GCC map </h2>

In [ ]:
import os
import glob
from matplotlib import cm
from PIL import Image, ImageOps 
from skimage import data, color
import subprocess
from skimage.transform import rescale, resize, downscale_local_mean

def find_hershel ():
    images_all = []
    masks_all = []
    pil_all =  []
    fmasks = []
    i = 0
    for key, v in filaments2.items ():
        original_path = 'datasets/colden/' + key + '.fits'     
        items = [k for k,v1 in v]
        angles_file = filaments [key][best_msims[i]][0]
        angle_fits = fits.open (angles_file)
        mask_fits = fits.open (items [best_msims[i]])
        fname = items [best_msims[i]].split('/')[-1]
        opts = [v1 for k,v1 in v]
        image_f = fits.open (original_path)
        print (original_path)
        image_d = np.array (image_f[0].data).astype('float32')
        image_dd = (image_d / image_d.max()) *  255.0;
        image_d = Image.fromarray(image_dd)
        newSize = (128,128)
        image_p = image_d.resize(newSize)
        image_f = np.array (image_p).astype ('float32')
        image_f = (image_f / image_f.max()) * 255.0;
        image_f = image_f.clip(min = 0)
        image_dd = image_f[image_f != 0]
        angles_d = np.array (angle_fits[0].data)
        mask_p = Image.fromarray(angles_d)
        mask_p = mask_p.resize(newSize)
        mask = np.array (mask_p)
        
        mask_d = np.array (mask_fits[0].data)
        mask_d = Image.fromarray(mask_d)
        mask_p = mask_d.resize(newSize)
        mask_real = np.array (mask_p)
        masks_all.append (mask)
        fmasks.append (mask_real)
        images_all.append (image_f)
        i+=1
    return images_all, masks_all, fmasks

In [ ]:
best_msims = find_best_msim_maps()

In [ ]:
images_all,angles_all,masks_all = find_hershel()

<h2> Reading HI dataset </h2>

In [ ]:
import os
import glob
from matplotlib import cm
from PIL import Image, ImageOps
from skimage import data, color
import subprocess
from skimage.transform import rescale, resize, downscale_local_mean

def find_h ():
    images_all1 = []
    masks_all1 = []
    pil_all =  []
    angles_all1 = []
    i = 0
    filament_names = list (set ([d.split ('\\')[1].split ('orig_lon')[1][0:-5].replace ('_K27_BAR7','').replace ('_smr27_wlen7','') for d in glob.glob ('datasets/Hdata/*orig_lon*')]))
    root = os.getcwd().replace ('\\','/') + '/'
    for filament in filament_names:
        filament.replace ('lat-','lat')
        mask_path = root + 'datasets/Hdata/suprht_theta_lon' + filament + '_K27_BAR7.fits'
        orig_path = root + 'datasets/Hdata/orig_lon' + filament + '.fits'
        mask2_path = root + 'datasets/Hdata/suprht_lon' + filament + '_K27_BAR7.fits'
        print (mask_path)
        if (os.path.exists (mask_path) == False):
            continue
        if (os.path.exists (orig_path) == False):
            continue
        if (os.path.exists (mask2_path) == False):
            print ('not found')
            continue
        mask = fits.open (mask_path)
        image_f = fits.open (orig_path)
        mask2 = fits.open (mask2_path)
        mask2 = np.array (mask2[0].data)
        image_d = np.array (image_f[0].data).astype('float32')
        image_dd = (image_d / image_d.max()) *  255.0;
        image_d = Image.fromarray(image_dd)
        newSize = (128,128)
        image_p = image_d.resize(newSize)
        image_f = np.array (image_p).astype ('float32')
        image_f = (image_f / image_f.max()) * 255.0;
        image_f = image_f.clip(min = 0)
        image_dd = image_f[image_f != 0]
        mask_d = np.array (mask[0].data)
        mask_p = Image.fromarray(mask_d)
        newSize = (128,128)
        mask_p = mask_p.resize(newSize)
        mask = np.array (mask_p)
        mask2 = Image.fromarray(mask2)
        newSize = (128,128)
        mask2 = mask2.resize(newSize)
        mask2 = np.array (mask2)
        masks_all1.append (mask2)
        images_all1.append (image_f)
        angles_all1.append (mask)
        i+=1
    return images_all1, masks_all1, angles_all1

In [ ]:
images_allH, masks_allH, angles_allH = find_h ()

In [ ]:
inmaps, anglemaps = find_masks_and_images_best()

<h1> Dataset preparation </h1>

<h3> Preparing input data with Planck-1 / Planck-cc / Herschel datasets </h3>

In [ ]:
images_all = np.array (images_all, dtype = np.float)
angles_all = np.array (angles_all, dtype=np.float)
masks_all = np.array (masks_all, dtype=np.float)
y_train2 = np.zeros ((images_all.shape[0] , images_all.shape[1],images_all.shape[2]))
print (y_train2.shape)
for i in range (images_all.shape[0]):
    y_train2 [i,:] = np.array(angles_all [i])
x_train = images_all[0:].reshape ((images_all.shape[0],images_all.shape[1],images_all.shape[2],1))
y_train = np.expand_dims (y_train2,axis = -1)

<h3> Preparing input data with HI dataset </h3>

In [ ]:
images_allH = np.array (images_allH, dtype = np.float)
angles_allH = np.array (angles_allH, dtype=np.float)
masks_allH = np.array (masks_allH, dtype=np.float)
y_train2 = np.zeros ((images_allH.shape[0] , images_allH.shape[1],images_allH.shape[2]))
for i in range (images_allH.shape[0]):
    y_train2 [i,:] = np.array(angles_allH [i])
x_train = masks_allH[0:].reshape ((images_allH.shape[0],images_allH.shape[1],images_allH.shape[2],1))
y_train = np.expand_dims (y_train2,axis = -1)

In [ ]:
x_test = masks_all[100:]
x_test = x_test.reshape ((x_test.shape[0],images_all.shape[1],images_all.shape[2],1))
y_test = y_train2[100:]
y_test = y_test.reshape ((y_test.shape[0],y_train2.shape[1],y_train2.shape[2],1))

<h1> Training a convolutional NN for the given dataset </h1>

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=configuration)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(121, 121, 1)),
  tf.keras.layers.AveragePooling2D((3, 3)),
  tf.keras.layers.Conv2D(32, (3,3)),  
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.AveragePooling2D((2, 2)),  
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),  
  tf.keras.layers.Flatten(),  
  tf.keras.layers.Dense (121 * 121)              
])
model.summary()
loss_fn = tf.keras.losses.MeanSquaredError()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['mse'])

history = model.fit(x_train, y_train, validation_split = 0.1, epochs=100)

<h1> U-net model </h1>

<h2> Dataset preparation </h2>

In [ ]:
x_train = x_train.reshape ((images_allH.shape[0],images_allH.shape[1] * images_allH.shape[2]))
y_train = y_train.reshape ((images_allH.shape[0],y_train2.shape[1] * y_train2.shape[2]))
scalerx = StandardScaler()
scalery = StandardScaler()
x_train = scalerx.fit_transform(x_train)
y_train = scalery.fit_transform(y_train)

In [ ]:
x_train = x_train.reshape ((images_allH.shape[0],images_allH.shape[1] * images_allH.shape[2]))
y_train = y_train.reshape ((images_allH.shape[0],y_train2.shape[1] * y_train2.shape[2]))
x_train = x_train.reshape ((images_allH.shape[0],images_allH.shape[1],images_allH.shape[2]))
y_train = y_train.reshape ((images_allH.shape[0],y_train2.shape[1],y_train2.shape[2]))
x_max = []
y_max = []
for i in range (images_allH.shape[0]):
    x_max.append (x_train [i].max())
for i in range (images_allH.shape[0]):
    y_max.append (y_train [i].max())
dimSize = 128
x_train3 = np.zeros ((30,dimSize*dimSize))
y_train3 = np.zeros ((30,dimSize*dimSize))
x_test3 = np.zeros ((7,dimSize*dimSize))
y_test3 = np.zeros ((7,dimSize*dimSize))
for i in range (30):
    x_train3[i,:] = x_train[i,:,:].reshape (dimSize * dimSize)
    y_train3[i,:] = y_train[i,:,:].reshape (dimSize * dimSize)
j = 0
for i in range (31,37):
    x_test3[j,:] = x_train[i,:,:].reshape (dimSize * dimSize)
    y_test3[j,:] = y_train[i,:,:].reshape (dimSize * dimSize)
    j += 1

In [ ]:
x_train = x_train.reshape ((images_all.shape[0],images_all.shape[1] * images_all.shape[2]))
y_train = y_train.reshape ((images_all.shape[0],y_train2.shape[1] * y_train2.shape[2]))
x_train_t = x_train.reshape ((images_all.shape[0],images_all.shape[1],images_all.shape[2]))
y_train_t = y_train.reshape ((images_all.shape[0],y_train2.shape[1],y_train2.shape[2]))
x_max = []
y_max = []
for i in range (images_all.shape[0]):
    x_max.append (x_train_t [i].max())
for i in range (images_all.shape[0]):
    y_max.append (y_train_t [i].max())
print (x_train_t.shape)
print (y_train_t.shape)
dimSize = 128
x_train3 = np.zeros ((100,dimSize*dimSize))
y_train3 = np.zeros ((100,dimSize*dimSize))

x_test3 = np.zeros ((37,dimSize*dimSize),dtype = np.float)
y_test3 = np.zeros ((37,dimSize*dimSize),dtype = np.float)

for i in range (100):
    x_train3[i,:] = x_train_t[i,:,:].reshape (dimSize * dimSize)
    y_train3[i,:] = y_train_t[i,:,:].reshape (dimSize * dimSize)
j = 0
for i in range (101,137):
    x_test3[j,:] = x_train_t[i,:,:].reshape (dimSize * dimSize)
    y_test3[j,:] = y_train_t[i,:,:].reshape (dimSize * dimSize)
    j += 1

In [ ]:
x_train = x_train.reshape ((images_all.shape[0],images_all.shape[1] * images_all.shape[2]))
y_train = y_train.reshape ((images_all.shape[0],y_train2.shape[1] * y_train2.shape[2]))
x_train = x_train.reshape ((images_all.shape[0],images_all.shape[1],images_all.shape[2]))
y_train = y_train.reshape ((images_all.shape[0],y_train2.shape[1],y_train2.shape[2]))
x_max = []
y_max = []
dimSize = 128
x_train3 = np.zeros ((100,dimSize*dimSize))
y_train3 = np.zeros ((100,dimSize*dimSize))

x_test3 = np.zeros ((15,dimSize*dimSize),dtype = np.float)
y_test3 = np.zeros ((15,dimSize*dimSize),dtype = np.float)

for i in range (100):
    x_train3[i,:] = x_train[i,:,:].reshape (dimSize * dimSize)
    y_train3[i,:] = y_train[i,:,:].reshape (dimSize * dimSize)
j = 0
for i in range (101,115):
    x_test3[j,:] = x_train[i,:,:].reshape (dimSize * dimSize)
    y_test3[j,:] = y_train[i,:,:].reshape (dimSize * dimSize)
    j += 1

<h1> Training U-net model </h1>

In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(optimizer = 'Adam', pretrained_weights = None,input_size = (128,128,1)):
    inputs = tf.keras.Input(input_size)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = tf.keras.layers.Dropout(0.25)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = tf.keras.layers.Dropout(0.25)(conv5)

    up6 = tf.keras.layers.Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = tf.keras.layers.concatenate([drop4,up6], axis = 3)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = tf.keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = tf.keras.layers.concatenate([conv3,up7], axis = 3)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = tf.keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = tf.keras.layers.concatenate([conv2,up8], axis = 3)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = tf.keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = tf.keras.layers.concatenate([conv1,up9], axis = 3)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = tf.keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = tf.keras.layers.Conv2D(1, 1, activation = 'linear')(conv9)

    model = tf.keras.Model(inputs, conv10)

    model.compile(optimizer = optimizer, loss = 'mse', metrics = ['mse'])
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=unet, epochs=70, batch_size=1, verbose=0)
optimizer = ['SGD', 'RMSprop', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize*dimSize)))

In [ ]:
lrs = []
initial_lr = 0.00001
for i in range (7):
    lrs.append (initial_lr)
    initial_lr = initial_lr * 0.1

In [ ]:
models = []
lrs = []
initial_lr = 0.00001
for i in range (7):
    lrs.append (initial_lr)
    initial_lr = initial_lr * 0.1
for i in range (7):
    model = unet()
    model.summary()
    loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
    opt = tf.keras.optimizers.Adam(learning_rate = lrs [i])
    model.compile(optimizer=opt,
                  loss=loss_fn,
                  metrics=['mse'])
    history = model.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=100)
    models.append (model)

In [ ]:
from IPython.core.history import HistoryAccessor
hista = HistoryAccessor()
z1 = hista.search('*images_all = np.array*')
z1.fetchall()

In [ ]:
plt.plot(history.history['val_loss'])

In [ ]:
print (x_train3[10].shape)
xd = x_train3[10].reshape (128,128)
plt.imshow (xd)

<h3> Finding best model hyper-parameters for HI </h3>

In [ ]:
model = unet()
model.summary()
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=150)

In [ ]:
model2.save('models/unet/')

In [ ]:
model8 = load_model ('C:/Users/NU/Documents/dana_collaboration_project/models/model_8_H_dataset/saved_model')

In [ ]:
model2 = unet()
model2.summary()
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)
model2.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])
history = model2.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=150)

In [ ]:
model3 = unet()
model3.summary()
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adamax(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax",
)
model3.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])
history = model3.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=150

In [ ]:
model4 = unet()
model4.summary()
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(
    learning_rate=0.00001#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)
model4.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])
history = model4.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=150)

In [ ]:
model5 = unet()
model5.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model5.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model5.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=150)


In [ ]:
model7 = unet()
model7.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.000001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model7.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model7.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model8 = unet()
model8.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model8.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model8.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model9 = unet()
model9.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model9.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model9.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=500)


In [ ]:
model10 = unet()
model10.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")
model10.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])
history = model10.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)

In [ ]:
model11 = unet()
model11.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model11.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model11.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model12 = unet()
model12.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model12.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model12.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model13 = unet()
model13.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.01,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model13.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model13.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model14 = unet()
model14.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.008,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model14.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model14.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model15 = unet()
model15.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model15.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model15.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h = unet()
model_h.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_h.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hh = unet()
model_hh.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_hh.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hh.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hhh = unet()
model_hhh.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_hhh.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hhh.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h4 = unet()
model_h4.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00003,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_h4.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h4.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h5 = unet()
model_h5.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.000001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_h5.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h5.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h6 = unet()
model_h6.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_h6.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h6.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h7 = unet()
model_h7.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

model_h7.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h7.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h8 = unet()
model_h8.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_h8.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h8.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h9 = unet()
model_h9.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.0001, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_h9.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h9.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h11.save ('models/model_adadelta_lr003.h5')
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adadelta_lr003.h5')
saved_m.summary()

In [ ]:
model_h10 = unet()
model_h10.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.05, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_h10.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h10.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h11 = unet()
model_h11.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.03, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_h11.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h11.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h12 = unet()
model_h12.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.04, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_h12.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h12.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h12.save ('models/model_adadelta_lr004.h5')
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adadelta_lr004.h5')
saved_m.summary()

In [ ]:
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adam_lr00005.h5')
saved_m.summary()

In [ ]:
model_h13 = unet()
model_h13.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 


opt = tf.keras.optimizers.Adamax(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)


model_h13.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h13.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h14 = unet()
model_h14.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 


opt = tf.keras.optimizers.Adamax(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_h14.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h14.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h16.save ('models/model_adamax_lr00001.h5')
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adamax_lr00001.h5')
saved_m.summary()

In [ ]:
model_h14 = unet()
model_h14.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.00005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_h14.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h14.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h14.save ('models/model_adamax_lr00005.h5')
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adamax_lr00005.h5')
saved_m.summary()

In [ ]:
model_h15 = unet()
model_h15.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.00005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_h15.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h15.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h16 = unet()
model_h16.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adam(learning_rate=0.00005)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax",)

model_h16.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h16.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h16.save ('models/model_adam_lr00005.h5')
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adam_lr00005.h5')
saved_m.summary()

In [ ]:
model_h17 = unet()
model_h17.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax",)

model_h17.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h17.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h17.save ('models/model_adam_lr0001.h5')

In [ ]:
model_h17.save ('models/model_adam_lr0001.h5')
saved_m = unet()
saved_m.summary()
saved_m.load_weights ('models/model_adam_lr0001.h5')
saved_m.summary()

In [ ]:
model_h18 = unet()
model_h18.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate=0.0002)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax",)

model_h18.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h18.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h19 = unet()
model_h19.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate=0.004)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax",)


model_h19.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h19.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


 <h3> Finding best model hyper-parameters for Herschel dataset </h3>

In [ ]:
model_hershel = unet()
model_hershel.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.042, rho=0.95, epsilon=1e-07, name="Adadelta"
)
model_hershel.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel1 = unet()
model_hershel1.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.00005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_hershel1.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel1.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel2 = unet()
model_hershel2.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_hershel2.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel2.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel3 = unet()
model_hershel3.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

model_hershel3.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel3.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel4 = unet()
model_hershel4.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00001)

model_hershel4.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel4.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel5 = unet()
model_hershel5.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.0004, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_hershel5.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel5.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel6 = unet()
model_hershel6.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.004, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_hershel6.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel6.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel7 = unet()
model_hershel7.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

model_hershel7.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel7.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel8 = unet()
model_hershel8.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00001)

model_hershel8.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel8.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel9 = unet()
model_hershel9.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.004,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_hershel9.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel9.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel10 = unet()
model_hershel10.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_hershel10.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel10.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel11 = unet()
model_hershel11.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adamax(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_hershel11.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel11.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel12 = unet()
model_hershel12.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_hershel12.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel12.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel12 = unet()
model_hershel12.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_hershel12.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel12.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel13 = unet()
model_hershel13.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0007)

model_hershel13.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel13.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel14 = unet()
model_hershel14.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.000007)

model_hershel14.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel14.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel15 = unet()
model_hershel15.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adamax(
    learning_rate=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

model_hershel15.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel15.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_hershel16 = unet()
model_hershel16.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.1,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_hershel16.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel16.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
import numpy as np
from keras.callbacks import LearningRateScheduler

def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule)

lr_sched = step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=2)

In [ ]:
model_hershel17 = unet()
model_hershel17.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0004)

model_hershel17.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_hershel17.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


<h3> Finding best model hyper-parameters for Planck dataset </h3>

In [ ]:
model_p = unet()
model_p.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00005)

model_p.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p = unet()
model_p.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_p.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p1 = unet()
model_p1.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.002)

model_p1.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p1.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p2 = unet()
model_p2.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_p2.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p2.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p3 = unet()
model_p3.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.RMSprop(
    learning_rate=0.00001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    name="RMSprop"
)
model_p3.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p3.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p4 = unet()
model_p4.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.RMSprop(
    learning_rate=0.00002,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    name="RMSprop"
)
model_p4.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p4.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p5 = unet()
model_p5.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_p5.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p5.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p6 = unet()
model_p6.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.00003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_p6.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p6.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p7 = unet()
model_p7.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.00003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_p7.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p7.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p8 = unet()
model_p8.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_p8.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p8.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p9 = unet()
model_p9.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00003)

model_p9.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p9.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p10 = unet()
model_p10.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00007)

model_p10.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p10.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p11 = unet()
model_p11.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00005)

model_p11.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p11.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p12 = unet()
model_p12.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0005)

model_p12.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p12.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p13 = unet()
model_p13.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.000001)

model_p13.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p13.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p14 = unet()
model_p14.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

model_p14.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p14.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p15 = unet()
model_p15.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

model_p15.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p15.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p16 = unet()
model_p16.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Nadam(
    learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

model_p16.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p16.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p17 = unet()
model_p17.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adadelta(
    learning_rate=0.0001, rho=0.95, epsilon=1e-07, name="Adadelta"
)

model_p17.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p17.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_p17 = unet()
model_p17.summary()
#Logarithmic
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.00003)

model_p17.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_p17.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_h20 = unet()
model_h20.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model_h20.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_h20.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam",
)

In [ ]:
model16 = unet()
model16.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.00001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model16.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model16.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model17 = unet()
model17.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.000001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model17.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model17.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_H = unet()
model_H.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adam(learning_rate=0.000003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax",)

model_H.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_H.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model_H3 = unet()
model_H3.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 
opt = tf.keras.optimizers.Adam(learning_rate = 0.000001)

model_H3.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model_H3.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
model18 = unet()
model18.summary()
loss_fn = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.SUM) # 

opt = tf.keras.optimizers.Adagrad(
     learning_rate=0.0000001,
     initial_accumulator_value=0.1,
     epsilon=1e-07,
     name="Adagrad")

model18.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])

history = model18.fit(x_train3.reshape ((x_train3.shape[0],dimSize,dimSize,1)), y_train3.reshape ((x_train3.shape[0],dimSize,dimSize)), validation_split = 0.1, epochs=300)


In [ ]:
map1 = fits.open ('figures/unet/theta_1.fits')
map2 = fits.open ('figures/unet/unet_1.fits')
map3 = fits.open ('figures/unet/intensity_1.fits')
map1 = map1[0].data
map2 = map2[0].data
map3 = map3[0].data

f, axarr = plt.subplots(1,3) 
axarr[0].axis('off')
axarr[0].imshow(map1, cmap = 'Wistia')
#axarr[0].set_title('rht theta')
plt.axis('off')
axarr[1].imshow(map2, cmap = 'Wistia')
axarr[1].axis('off')
axarr[2].imshow( map3, cmap = 'RdBu_r')
axarr[2].axis('off')
#axarr[2].set_title('intensity map')
plt.savefig('figures/unet/figure5/map_paper' + '.png',
        bbox_inches ="tight",
        pad_inches = 1,
        facecolor ="w",
        edgecolor ='w',
        dpi = 300,    
        orientation ='landscape')

<h2> Test a model on a sample map G202.85 from Herschel dataset </h2>

In [ ]:
mp = fits.open ('example_map.fits')
map_data = mp[0].data
xd = np.expand_dims(map_data.reshape (dimSize,dimSize,1), axis=0)
data = model4.predict(xd)
plt.show (data)